In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import json

from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)

past_last_date = False

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = []
page_num = 0

ticker = 'msft'

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)


while True:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://news.microsoft.com/category/press-releases/page/{page_num}/', headers=headers)
    

    soup = BeautifulSoup(html.content) 
    content = soup.find('section', attrs={'content-area'})
    if content == None:
        break

    articles = content.find_all('article')
    
    for article in articles:
        date = article.find('div', attrs={'c-paragraph-3 c-meta-text'}).text.lstrip().rstrip() 
        title = article.find('a').text.lstrip().rstrip() 
        
        url = article.find('a').get('href')
        
        date = date.replace(',', '')    
        date = datetime.strptime(date, '%B %d %Y').date()

        content = get_content_header(url, 'meta' , {'property': 'og:description'})

        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break        
    
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    if past_last_date == True: break    
    page_num = page_num + 1
    
save_data(data, ticker, sector)

past_last_date = False

In [6]:
import urllib.request

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

headers={'User-Agent':user_agent,} 

request = urllib.request.Request(url, None, headers) #The assembled request

In [14]:
type(request.getcode())

AttributeError: 'Request' object has no attribute 'getcode'

In [12]:
response = urllib.request.urlopen(request)

HTTPError: HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Permanently